In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.33.2 --progress-bar off
!pip install -qqq langchain==0.0.299 --progress-bar off
!pip install -qqq chromadb==0.4.10 --progress-bar off
!pip install -qqq xformers==0.0.21 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq tokenizers==0.14.0 --progress-bar off
!pip install -qqq optimum==1.13.1 --progress-bar off
!pip install -qqq auto-gptq==0.4.2 --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --progress-bar off
!pip install -qqq unstructured==0.10.16 --progress-bar off
!pip install pdfminer.six
!pip install tiktoken
!pip install -U faiss-cpu 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.2.3 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.6.4 requires pydantic>=2, but you have pydantic 1.10.17 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.20.0 requires huggingface-hub>=0.21.2, but you have huggingfac

In [2]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain.document_loaders import PDFMinerLoader
from transformers import GPT2TokenizerFast
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain import PromptTemplate

2024-07-19 01:30:18.908928: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-19 01:30:18.909045: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-19 01:30:19.092056: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
MODEL_NAME = "TheBloke/Llama-2-13B-chat-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

In [5]:
question = "Explain what is the major cause of landslide in Hong Kong? Summerised some of the point towards landslide."

In [6]:
loader = PDFMinerLoader("/kaggle/input/csdidata/landslide2020georeport.pdf")
docs = loader.load()
text_splitter = SentenceTransformersTokenTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)
query_result = embeddings.embed_query(texts[0].page_content)
db = FAISS.from_documents(texts, embeddings)
results = db.similarity_search(question, k=20)
print(results[0].page_content)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)


.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

were prepared by the respective departments. further details of these landslide incidents can be found in the relevant files of the three district divisions and lpm division 2 of the geo. wherever possible, the dates and times of the landslides were assessed by geotechnical professionals. of the 214 landslides, the timing of occurrence was determined to within one day for 48 incidents based on the reported date of failure given in the incident reports. for the remaining landslide incidents, the timing of occurrence could not be ascertained due to lack of information or that the incidents were not reported to the geo or other government departments until several days or even weeks after occurrence. 3. 2 consequence of landslides the consequence of landslides in terms of the types of facilities affected ( e. g. buildings, registered squatter dwellings, roads, catchwaters, car parks, etc. ) in different regions is summarised in table 3. 2. in regard to the landslides with significant cons

In [7]:
template = """
<s>[INST] <<SYS>>
Use the following information to answer the question at the end.
<</SYS>>

{context}

{question} [/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])


qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

result = qa_chain("Explain what is the major cause of landslide in Hong Kong? Summerised some of the point towards landslide.")
print(result["result"].strip())

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Based on the information provided in the report, the major cause of landslides in Hong Kong is heavy rainfall. The report states that "the majority of the landslides occurred during or immediately after periods of heavy rainfall." Additionally, the report highlights that the rainfall intensity and duration were the primary factors contributing to the landslides.

Some of the key points summarized in the report regarding the causes of landslides are:

1. Heavy rainfall: The report emphasizes that heavy rainfall was the main factor causing landslides in Hong Kong.
2. Rainfall intensity and duration: The report notes that both the intensity and duration of rainfall played a crucial role in triggering landslides.
3. Soil moisture: The report suggests that soil moisture levels were a critical factor, as saturated soils were more susceptible to landslides.
4. Slope angle and orientation: The report mentions that steep slopes and those facing upslope were more prone to landslides.
5. Ground c

In [8]:
question = "Which river is the most dirtiest river quality in Hong Kong, list out their major pollutants."

In [9]:
loader = PDFMinerLoader("/kaggle/input/csdidata/riverreport22.pdf")
docs = loader.load()
text_splitter = SentenceTransformersTokenTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)
query_result = embeddings.embed_query(texts[0].page_content)
db = FAISS.from_documents(texts, embeddings)
results = db.similarity_search(question, k=20)
print(results[0].page_content)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

river water quality in hong kong in 2022 j > l river water quality in hong kong in 2022 1 environmental protection department the government of the hong kong special administrative region mission to conduct a comprehensive and scientific monitoring programme that helps safeguard the health of hong kong ’ s rivers and streams and achieve the water quality objectives. disclaimer whilst the government of the hong kong special administrative region ( " the government " ) endeavours to ensure the accuracy of the information in this report, the government ( including its officers and employees ) makes no warranty, statement or representation, expressed or implied, with respect to accuracy, completeness, or usefulness of the information contained herein, and in so far as permitted by law, shall not have any legal liability or responsibility ( including liability for negligence ) for any loss, damage, injury and death which may result, whether directly or indirectly, from the supply or use of 

In [10]:
template = """
<s>[INST] <<SYS>>
Use the following information to answer the question at the end.
<</SYS>>

{context}

{question} [/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])


qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

result = qa_chain("Which river is the most dirtiest river quality in Hong Kong, list out their major pollutants.")
print(result["result"].strip())

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Based on the information provided in the report, the water quality of the rivers in Hong Kong was monitored in 2022, and the results are summarized in Figure 4. According to the map, the water quality of the rivers in Hong Kong varies, with some areas having excellent water quality, while others have poor water quality.

The most polluted river in Hong Kong is the Shing Mun River, which flows through the northern part of the territory. The water quality of this river is rated as "very bad" according to the Water Quality Index (WQI), with high levels of pollutants such as Ammonia Nitrogen, Chemical Oxygen Demand, Dissolved Oxygen, and Suspended Solids. These pollutants are primarily caused by human activities such as industrial waste and sewage discharge, as well as agricultural runoff.

Other heavily polluted rivers in Hong Kong include the Tai Po Kau Stream, the Lam Tsuen River, and the Tung Tze Stream, all of which are located in the northern parts of the territory. These rivers also

In [11]:
question = "Disucss the spatial distribution of the subdivided flat in Hong Kong and list out their changes from 2016 to 2021."

In [14]:
loader = PDFMinerLoader("/kaggle/input/csdidata/21c-SDUs.pdf")
docs = loader.load()
text_splitter = SentenceTransformersTokenTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)
query_result = embeddings.embed_query(texts[0].page_content)
db = FAISS.from_documents(texts, embeddings)
results = db.similarity_search(question, k=20)
print(results[0].page_content)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

size, 2016 and 2021 2016 年 [UNK] 2021 年 [UNK] 有 [UNK] [UNK] [UNK] [UNK] 5 年 [UNK] [UNK] [UNK] 部 [UNK] [UNK] [UNK] 分 的 [UNK] [UNK] 分 間 [UNK] 宇 [UNK] [UNK] 的 [UNK] [UNK] [UNK] 目 table 6. 9 households living in subdivided units by whether having internally migrated within the past 5 years, 2016 and 2021 40 41 42 [UNK] 6. 10 2016 年 [UNK] 2021 年 [UNK] [UNK] [UNK] 成 [UNK] 中 有 [UNK] [UNK] 地 [UNK] [UNK] 定 [UNK] [UNK] [UNK] 7 年 人 士 [UNK] 分 的 [UNK] [UNK] 分 間 [UNK] 宇 [UNK] [UNK] 的 [UNK] [UNK] [UNK] 目 table 6. 10 households living in subdivided units 43 by whether having any household members from the mainland having resided in hong kong for less than 7 years, 2016 and 2021 2021 年 人 口 [UNK] [UNK] 主 [UNK] [UNK] [UNK] [UNK] ： [UNK] [UNK] 分 間 [UNK] 宇 [UNK] [UNK] 人 士 香 [UNK] [UNK] [UNK] 行 政 [UNK] 政 [UNK] [UNK] [UNK] [UNK] v 2021 population census thematic report : persons living in subdivided units census and statistics department, hong kong special administrative region [UNK] [UNK] [UNK] 一 [UNK] list

In [15]:
template = """
<s>[INST] <<SYS>>
Use the following information to answer the question at the end.
<</SYS>>

{context}

{question} [/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])


qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

result = qa_chain("Disucss the spatial distribution of the subdivided flat in Hong Kong and list out their changes from 2016 to 2021.")
print(result["result"].strip())

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Based on the information provided in the table, we can see that the spatial distribution of subdivided flats in Hong Kong has changed significantly from 2016 to 2021. In 2016, the majority of subdivided flats were located in Kowloon (62.4%), followed by Hong Kong Island (13.7%), and New Territories (24.0%). However, by 2021, there had been a noticeable shift in the geographical distribution of subdivided flats, with a significant increase in the number of flats in New Territories (45.0%) and a decrease in the number of flats in Hong Kong Island (10.4%).

Moreover, the data also shows that the proportion of subdivided flats in each region has changed over time. For instance, in 2016, the proportion of subdivided flats in Kowloon was 62.4%, while in 2021 it had decreased to 50.3%. Similarly, the proportion of subdivided flats in Hong Kong Island increased from 13.7% in 2016 to 20.5% in 2021.

Overall, these changes suggest that the spatial distribution of subdivided flats in Hong Kong ha

In [17]:
question = "Which area has the most dirtiest sea water in Hong Kong, suggest some clean water areas for water activities."

In [18]:
loader = PDFMinerLoader("/kaggle/input/csdidata/marinereport22.pdf")
docs = loader.load()
text_splitter = SentenceTransformersTokenTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)
query_result = embeddings.embed_query(texts[0].page_content)
db = FAISS.from_documents(texts, embeddings)
results = db.similarity_search(question, k=20)
print(results[0].page_content)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

e e e e e e e e e e e e e e e e e e e e e e e note : 1. all locations are based on wgs84 datum 2. water quality and sediment monitoring stations in typhoon shelters are marked with an asterisk * marine water quality in hong kong in 2022 depth ( m ) approx. 4 7 8 4 12 11 22 6 14 14 33 11 8 14 8 8 5 8 7 6 12 21 24 7 6 6 6 8 13 6 11 17 20 15 21 3 6 5 10 16 2 2 3 4 8 34 11 14 20 8 8 4 6 11 16 18 20 12 13 31 28 25 24 22 17 28 35 13 20 26 7 10 16 21 21 6 12 38 12 8 12 11 8 14 10 11 14 11 11 5 5 6 5 5 6 5 a - 5 i m a r n e w a t e r q u a l i t y i n h o n g k o n g i n 2 0 2 2 bathing beaches and secondary contact recreation subzones in hong kong in 2022 113° 50 ’ e 114° 00 ’ e 114° 10 ’ e 114° 20 ’ e 114° 30 ’ e shenzhen n 22° 30 ’ n 22° 20 ’ n new territories 22° 30 ’ n kowloon 22° 20 ’ n lantau island hong kong island a - 6 22° 10 ’ n 113° 50 ’ e 0 5 10 km gazetted beach non - gazetted beach secondary contact recreation 22° 10 ’ n subzone ( area : approx. 260km2 ) 114° 00 ’ e 114° 10 ’ e 

In [19]:
template = """
<s>[INST] <<SYS>>
Use the following information to answer the question at the end.
<</SYS>>

{context}

{question} [/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])


qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

result = qa_chain("Which area has the most dirtiest sea water in Hong Kong, suggest some clean water areas for water activities.")
print(result["result"].strip())

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Based on the information provided, it appears that the eastern waters of Hong Kong have the poorest water quality, specifically the Mirs Bay Water Control Zone, which had a relatively low overall marine water quality compliance rate of 96% in 2022. The water quality is affected by high levels of nutrients, low dissolved oxygen, and high temperatures, which can lead to harmful algal blooms and poor aquatic life.

On the other hand, the central, western, and southern waters of Hong Kong generally have better water quality, with higher compliance rates for marine water quality standards. Some of the cleaner water areas for water activities include:

1. Western waters: These waters have consistently achieved high compliance rates for marine water quality standards, with low levels of nutrients and high dissolved oxygen. Popular water activities in this area include swimming, kayaking, and stand-up paddleboarding.
2. Southern waters: These waters are known for their pristine condition, with